In [1]:
!python -V

Python 3.9.13


In [10]:
!pwd

/Users/yothinpu/mlops-zoomcamp/03-experiment-tracking


In [2]:
import pandas as pd

In [4]:
import sklearn
print(sklearn.__version__)

1.1.2


In [3]:
import pickle

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [7]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyx-taxi-experiment")

2022/11/04 18:30:11 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2022/11/04 18:30:11 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='./mlruns/1', creation_time=1667561412033, experiment_id='1', last_update_time=1667561412033, lifecycle_stage='active', name='nyx-taxi-experiment', tags={}>

In [11]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'sqlite:///mlflow.db'


In [9]:
mlflow.list_experiments()

/var/folders/l2/3m75q01s57qdd9zxly800t5r0000gn/T/ipykernel_538/546495964.py:1: FutureWarning: ``mlflow.tracking.fluent.list_experiments`` is deprecated. This method will be removed in a future release. Use ``search_experiments()`` instead.
  mlflow.list_experiments()
/Users/yothinpu/mlops-zoomcamp/env/lib/python3.9/site-packages/mlflow/tracking/fluent.py:1065: FutureWarning: ``mlflow.tracking.client.MlflowClient.list_experiments`` is deprecated. This method will be removed in a future release. Use ``search_experiments()`` instead.
  return MlflowClient().list_experiments(


[<Experiment: artifact_location='./mlruns/0', creation_time=1667561412027, experiment_id='0', last_update_time=1667561412027, lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', creation_time=1667561412033, experiment_id='1', last_update_time=1667561412033, lifecycle_stage='active', name='nyx-taxi-experiment', tags={}>]

In [17]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [18]:
df_train = read_dataframe('green_tripdata_2021-01.parquet')
df_val = read_dataframe('green_tripdata_2021-02.parquet')

In [19]:
len(df_train), len(df_val)

(73908, 61921)

In [20]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [21]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [22]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
train_dicts = df[categorical + numerical].to_dict(orient='records') #

dv = DictVectorizer() #
X_train = dv.fit_transform(train_dicts) #

target = 'duration'
y_train = df[target].values

In [23]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

5.699564118198945

In [ ]:
sns.distplot(y_pred, label='prediction')
sns.distplot(y_train, label='actual')

plt.legend()

In [25]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [27]:
lr = Lasso(0.1)
lr.fit(X_train,y_train)
y_pred = lr.predict(X_val)

mean_squared_error(y_val,y_pred,squared=False)

12.143233939663894

In [28]:
with mlflow.start_run():

    mlflow.set_tag("developer", "yothin")#

    mlflow.log_param("train-data-path", "green_tripdata_2021-01.parquet")#
    mlflow.log_param("valid-data-path", "green_tripdata_2021-02.parquet")#

    alpha = 0.1
    mlflow.log_param("alpha", alpha) #
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)#

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")#

# Predict 

In [8]:
# dependencies
import pickle
import pandas as pd
from sklearn.metrics import mean_squared_error
from read_data import *

# load model
with open('models/lin_reg.bin', 'rb') as f_out:
    read_model = pickle.load(f_out)

#read and transform 

test_df = read_dataframe('green_tripdata_2021-03.parquet')
test_df['PU_DO'] = test_df['PULocationID'] + '_' + test_df['DOLocationID']
categorical = ['PU_DO']
numerical = ['trip_distance'] 
dv = read_model[0] # load from model pickle
test_dicts = test_df[categorical + numerical].to_dict(orient='records')
X_new = dv.transform(test_dicts)

# load and predict 
y_pred = read_model[1].predict(X_new)

# target for model scoring

test_df['duration'] = test_df.lpep_dropoff_datetime - test_df.lpep_pickup_datetime
test_df.duration = test_df.duration.apply(lambda td: td.total_seconds() / 60)
target = 'duration' # label
y_new_real = test_df[target].values
mean_squared_error(y_new_real, y_pred, squared=False)

7.720735231839729

In [ ]:
# try Ridge , Lasso 